In [40]:
pip install accelerate -U evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 18.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset
from huggingface_hub import notebook_login

In [3]:
notebook_login()

In [4]:
checkpoint = "bert-base-uncased"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
dataset = load_dataset("mwarchalowski/names-data")

In [7]:
dataset = load_dataset("mwarchalowski/names-data", split="train[:10%]")

In [8]:
dataset = dataset.filter(lambda e: e["name"] is not None)

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["name"], truncation=True)
    

In [10]:
dataset

Dataset({
    features: ['name', 'label'],
    num_rows: 468400
})

In [11]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [13]:
samples = tokenized_datasets[:8]
samples.items()

dict_items([('name', ['Murphy Facility Solutions', 'Mansell Building Solutions', 'Johnny Boulos', 'Gurdeep Singh', 'Stefan Birrer', 'ECG Egineering Consulting Group', 'Topdeck Travel', 'Tamar Stiebel']), ('label', [0, 0, 1, 1, 1, 0, 0, 1]), ('input_ids', [[101, 7104, 4322, 7300, 102], [101, 16042, 5349, 2311, 7300, 102], [101, 5206, 8945, 18845, 2015, 102], [101, 19739, 25547, 13699, 5960, 102], [101, 8852, 12170, 14343, 2099, 102], [101, 14925, 2290, 1041, 11528, 20550, 10552, 2177, 102], [101, 2327, 26547, 3604, 102], [101, 17214, 2906, 2358, 2666, 8671, 102]]), ('token_type_ids', [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]), ('attention_mask', [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]])])

In [14]:
samples = {k: v for k, v in samples.items() if k not in ["name", "label"]}

In [15]:
batch = data_collator(samples)
batch

{'input_ids': tensor([[  101,  7104,  4322,  7300,   102,     0,     0,     0,     0],
        [  101, 16042,  5349,  2311,  7300,   102,     0,     0,     0],
        [  101,  5206,  8945, 18845,  2015,   102,     0,     0,     0],
        [  101, 19739, 25547, 13699,  5960,   102,     0,     0,     0],
        [  101,  8852, 12170, 14343,  2099,   102,     0,     0,     0],
        [  101, 14925,  2290,  1041, 11528, 20550, 10552,  2177,   102],
        [  101,  2327, 26547,  3604,   102,     0,     0,     0,     0],
        [  101, 17214,  2906,  2358,  2666,  8671,   102,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0,

In [16]:
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 9]),
 'token_type_ids': torch.Size([8, 9]),
 'attention_mask': torch.Size([8, 9])}

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer", fp16=True, per_device_train_batch_size=600)


In [18]:
from transformers import AutoModelForSequenceClassification

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
train_validation_set = tokenized_datasets.train_test_split(test_size=0.1)
train_validation_set = train_validation_set.remove_columns(["name"])
train_validation_set.rename_column("label", "labels")

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 421560
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 46840
    })
})

In [21]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_validation_set["train"],
    eval_dataset=train_validation_set["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
trainer.train()


Step,Training Loss
500,0.071100
1000,0.048600
1500,0.039100
2000,0.030300


TrainOutput(global_step=2109, training_loss=0.04620340979455376, metrics={'train_runtime': 216.2388, 'train_samples_per_second': 5848.535, 'train_steps_per_second': 9.753, 'total_flos': 1.13513509280736e+16, 'train_loss': 0.04620340979455376, 'epoch': 3.0})

In [31]:
predictions = trainer.predict(train_validation_set["test"])

In [45]:
print(predictions.predictions.shape, predictions.label_ids.shape)

(46840, 2) (46840,)


In [34]:
predictions

PredictionOutput(predictions=array([[-2.8476562,  3.1875   ],
       [ 4.53125  , -4.7890625],
       [ 2.0742188, -2.6210938],
       ...,
       [ 4.5625   , -4.9296875],
       [-2.7910156,  3.0742188],
       [-3.1835938,  3.5039062]], dtype=float32), label_ids=array([1, 0, 0, ..., 0, 1, 1]), metrics={'test_loss': 0.05594101920723915, 'test_runtime': 56.0663, 'test_samples_per_second': 835.439, 'test_steps_per_second': 104.43})

In [35]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [53]:
preds

array([1, 0, 0, ..., 0, 1, 1])

In [54]:
predictions.label_ids

array([1, 0, 0, ..., 0, 1, 1])

In [93]:
z = zip(preds, predictions.label_ids)

In [94]:
pos = [x for x,y in z if x == y]

In [90]:
len(negs)

675

In [92]:
len(pos)

0